# 🤖 RAG ChatBot 데모 (Demonstration)

이 노트북은 구축된 RAG 시스템의 전체 파이프라인(검색 -> 리랭킹 -> 생성)을 시연하고 성능 평가 결과를 확인하기 위해 작성되었습니다.

### 📋 목차
1. **시스템 초기화**: 환경 설정 및 모델 로드
2. **검색 품질 확인**: 질문에 대해 어떤 문서를 찾아오는지 확인 (Hybrid Search + Re-ranking)
3. **답변 생성 테스트**: 실제 질의응답 (QA) 시연
4. **정량 평가 결과**: ROUGE 점수 등 평가 지표 시각화

In [ ]:
import sys
import os
import pandas as pd
from IPython.display import display, Markdown

# 프로젝트 루트 경로를 시스템 경로에 추가 (모듈 임포트용)
sys.path.append(os.path.abspath(".."))

from src.common.config import config
from src.generation.rag import RAGChain
from src.indexing.vector_store import VectorStoreWrapper

## 1. 시스템 초기화 (System Initialization)
설정 파일(`config/local.yaml`)을 읽고, 벡터 데이터베이스(`ChromaDB`)와 RAG 체인을 로드합니다.

In [ ]:
# 벡터 저장소 초기화
print("Initializing Vector Store...")
vector_store = VectorStoreWrapper(config)
vector_store.initialize()

# RAG 체인 로드 (검색기 + GPT-5)
print("Loading RAG Chain...")
rag_chain = RAGChain(config=config, vector_store_wrapper=vector_store)
print("✅ System Ready!")

## 2. 검색 품질 확인 (Retrieval Quality)
LLM에게 답변을 생성시키기 전, **검색 단계**에서 올바른 문서가 조회되는지 확인합니다.
우리의 시스템은 다음 단계를 거칩니다:
1. **Hybrid Search**: 키워드(BM25) + 의미(Vector) 검색
2. **Re-ranking**: FlashRank를 이용해 관련성 순으로 재정렬

In [ ]:
query = "용인시 공고의 예산은 얼마야?"

print(f"🔍 검색 질문: {query}")
docs = rag_chain.retriever.invoke(query)

print(f"\n📄 검색된 문서 ({len(docs)}개):")
for i, doc in enumerate(docs):
    source = doc.metadata.get('source', 'Unknown')
    page = doc.metadata.get('page', '?')
    print(f"{i+1}. [{source} (p.{page})] {doc.page_content[:100]}...")

## 3. 답변 생성 테스트 (Generation)
검색된 문서를 Context로 삼아, GPT-5가 최종 답변을 생성합니다.

In [ ]:
answer = rag_chain.generate_answer(query, docs)

display(Markdown(f"### 🤖 AI 답변:\n{answer}"))

## 4. 정량 평가 결과 (Evaluation Results)
`data/eval_results.json`에 저장된 평가 결과를 로드하여 데이터프레임으로 시각화합니다.
ROUGE 점수는 생성된 요약문의 품질을 측정하는 지표입니다.

In [ ]:
eval_path = "../data/eval_results.json"

if os.path.exists(eval_path):
    df = pd.read_json(eval_path)
    display(df.head())
    
    print(f"\n📊 평균 ROUGE-L 점수: {df['rouge_l'].mean():.4f}")
else:
    print("평가 결과 파일이 없습니다. entrypoint/evaluate.py를 먼저 실행해주세요.")